In [1]:
import sys
sys.path[0]+='/..'
from dataset import GraphDataset
import torch
import torch_geometric
import os 
from model.gcn import GNNPolicy
import time
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
#set folder
# month=1
uc_model = '3bin'
instance = '50_0_1_w'
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")



In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =1000
BATCH_SIZE = 8
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            # print(len(batch.variable_features[torch.isnan(batch.variable_features)]))
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
            )
            # compute loss
            
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            loss = loss_fn(presol, weighted_sol)
            
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [5]:
for month in range(3,11):
    model_save_path = f'../model_save/{uc_model}_{instance}_{month}/'
    if not os.path.isdir(model_save_path):
        os.mkdir(model_save_path)
    data = GraphDataset(f'../newdatasets/{instance}/{uc_model}/train/{month}/')

    train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

    train_iter = torch_geometric.loader.DataLoader(
        train_data, batch_size = BATCH_SIZE, shuffle = True,
        num_workers = NUM_WORKERS, pin_memory=True)
    valid_iter = torch_geometric.loader.DataLoader(
        valid_data, batch_size = BATCH_SIZE, shuffle = False,
        num_workers = NUM_WORKERS)
    log_file = open(f'{model_save_path}train.log', 'wb')
    PredictModel = GNNPolicy().to(DEVICE)
    optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
    best_val_loss = 0.2
    cur_epoch = 0
    for epoch in range(NB_EPOCHS):
        begin=time.time()
        cur_epoch+=1
        train_loss = train(PredictModel, train_iter, optimizer)
        print(f"Month:{month} Epoch:{epoch} Train loss: {train_loss:0.6f}",end='\t')
        valid_loss = train(PredictModel, valid_iter, None)
        print(f"Month:{month} Epoch:{epoch} Valid loss: {valid_loss:0.6f}")
        if valid_loss<best_val_loss:
            cur_epoch=0
            best_val_loss = valid_loss
            print(best_val_loss)
            torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
        torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
        #100个epochloss不下降就停止
        if cur_epoch>100 and valid_loss>best_val_loss:
            break
        date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
        st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
        log_file.write(st.encode())
        log_file.flush()
        scheduler.step()
    print('done')

Month:3 Epoch:0 Train loss: 0.657684	Month:3 Epoch:0 Valid loss: 0.549502
Month:3 Epoch:1 Train loss: 0.381709	Month:3 Epoch:1 Valid loss: 0.320802
Month:3 Epoch:2 Train loss: 0.296815	Month:3 Epoch:2 Valid loss: 0.284937
Month:3 Epoch:3 Train loss: 0.241661	Month:3 Epoch:3 Valid loss: 0.227016
Month:3 Epoch:4 Train loss: 0.217260	Month:3 Epoch:4 Valid loss: 0.208449
Month:3 Epoch:5 Train loss: 0.214539	Month:3 Epoch:5 Valid loss: 0.217008
Month:3 Epoch:6 Train loss: 0.204388	Month:3 Epoch:6 Valid loss: 0.218275
Month:3 Epoch:7 Train loss: 0.194948	Month:3 Epoch:7 Valid loss: 0.188630
0.18863010082555853
Month:3 Epoch:8 Train loss: 0.194271	Month:3 Epoch:8 Valid loss: 0.185450
0.1854504301496174
Month:3 Epoch:9 Train loss: 0.183498	Month:3 Epoch:9 Valid loss: 0.189661
Month:3 Epoch:10 Train loss: 0.180963	Month:3 Epoch:10 Valid loss: 0.193947
Month:3 Epoch:11 Train loss: 0.175792	Month:3 Epoch:11 Valid loss: 0.170931
0.17093104642370474
Month:3 Epoch:12 Train loss: 0.251044	Month:3 Epo